<a href="https://colab.research.google.com/github/sheon-j/nlp-study/blob/main/tensorflow-nlp-tutorial/week06_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Study: Week 6 - Word Embedding

[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155) 스터디

---

**Contents**
8. 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)
9. 엘모(Embeddings from Language Model, ELMo)
10. 임베딩 벡터의 시각화(Embedding Visualization)

## 8. 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)

* **케라스의 임베딩 층(embedding layer)** 과 **사전 훈련된 워드 임베딩(pre-trained word embedding)**  비교

  1. 훈련 데이터를 케라스의 Embedding 메소드를 통해서 임베딩 층을 직접 구현하고 학습
  2. 사전 훈련된 워드 임베딩(Word2vec, FastText, GloVe)을 통해서<br>훈련 데이터와 매칭되는 임베딩 벡터를 학습

### 8.1. 케라스 임베딩 층(Keras Embedding layer)

* Embedding()
  * 훈련 데이터의 단어들에 대해 워드 임베딩을 수행하는 도구
  * 인공 신경망 구조 관점에서 임베딩 층(embedding layer)을 구현

#### (1) 임베딩 층은 룩업 테이블이다.

<div align="center">

![img](https://wikidocs.net/images/page/33793/lookup_table.PNG)

</div>

* 임베딩 층의 입력은 정수 인코딩 된 시퀀스
* 단어 → 단어에 매핑된 고유한 정수 값 → 임베딩 층 통과 → 밀집 벡터 
  * 임베딩 층은 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑. 이는 임베딩 벡터라고도 함
  * 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련
  * 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트

* 원-핫 벡터 vs 임베딩 
  * 원-핫 벡터
    * tokenizer.texts_to_sequences() 정수 인코딩 >>  to_categorical() 원 핫 인코딩
    * 단어의 개수가 늘어날 수록, 벡터를 저장하기 위해 필요한 공간이 계속 늘어난다는 단점
    * 벡터 표현의 공간적 낭비(희소 표현)를 해결한 것이 워드 임베딩의 밀집 표현



* 케라스의 임베딩 층의 기본 구현 코드

  ```python
  vocab_size = 20000
  output_dim = 128
  input_length = 500
  
  v = Embedding(vocab_size, output_dim, input_length=input_length)
  ```

  * 임베딩 층의 인자
    * **vocab_size =** 텍스트 데이터의 전체 단어 집합의 크기
    * **output_dim =** 워드 임베딩 후의 임베딩 벡터의 차원
    * **input_length =** 입력 시퀀스의 길이

  * 입력층
    * Embedding()은 (number of samples, input_length)인 정수 인코딩이 된 2D 텐서를 입력
    * Embedding()은 워드 임베딩 작업을 수행 (number of samples, input_length, embedding word dimentionality)
    * 3D 실수 텐서를 리턴 

#### (2) 임베딩 층 사용하기

* 목표: 문장의 긍, 부정을 판단하는 감성 분류 모델

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
# 긍정은 레이블 1, 부정은 레이블 0
y_train = [1, 0, 0, 1, 1, 0, 1]

In [2]:
# 케라스의 토크나이저를 사용하여 단어 집합을 만들고 그 크기를 확인
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print('단어 집합 :',vocab_size)

단어 집합 : 16


In [3]:
# 각 문장에 대해서 정수 인코딩을 수행
X_encoded = tokenizer.texts_to_sequences(sentences)
print('정수 인코딩 결과 :',X_encoded)

정수 인코딩 결과 : [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [4]:
# 가장 길이가 긴 문장의 길이
max_len = max(len(l) for l in X_encoded)
print('최대 길이 :',max_len)

최대 길이 : 4


In [5]:
# 최대 길이로 모든 샘플에 대해서 패딩을 진행
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print('패딩 결과 :')
print(X_train)

패딩 결과 :
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [8]:
# 이진 분류 모델을 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
# 임베딩 층 사용
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
# 출력층에 1개의 뉴런을 배치. 활성화 함수는 시그모이드 함수
model.add(Dense(1, activation='sigmoid'))

# 손실 함수로 binary_crossentropy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# 학습 과정에서 현재 각 단어들의 임베딩 벡터들의 값은 출력층의 가중치와 함께 학습
# 100 에포크 학습
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 495ms/step - loss: 0.6951 - acc: 0.5714
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6939 - acc: 0.5714
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6926 - acc: 0.5714
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6913 - acc: 0.5714
Epoch 5/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6900 - acc: 0.5714
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6887 - acc: 0.5714
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6874 - acc: 0.7143
Epoch 8/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6862 - acc: 0.8571
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6849 - acc: 0.8571
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6836 - acc: 0.8571
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 0

### 8.2. 사전 훈련된 워드 임베딩(Pre-Trained Word Embedding) 사용하기

* 이미 훈련되어져 있는 워드 임베딩을 가져와서 이를 임베딩 벡터로 사용
  * 훈련 데이터가 적은 상황
  * 해당 문제에 특화된 것은 아니지만 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용
  * 성능의 개선을 가져옴

* 사전 훈련된 GloVe와 Word2Vec 임베딩을 사용해서 모델을 훈련시키는 실습
  * GloVe [다운로드](http://nlp.stanford.edu/data/glove.6B.zip)
  * Word2Vec [다운로드](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM )

In [9]:
# 앞서 사용한 훈련 데이터
print(X_train)
print(y_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]
[1 0 0 1 1 0 1]


#### (1) 사전 훈련된 GloVe 사용하기

In [10]:
# glove.6B.zip 다운 
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall() 
zf.close()

In [11]:
# glove.6B.zip => glove.6B.100d.txt에 있는 모든 임베딩 벡터 로드 후 확인
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]

    # 100개의 값을 가지는 array로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [12]:
# 임의의 단어 'respectable'의 임베딩 벡터값과 크기를 출력
print(embedding_dict['respectable'])
print('벡터의 차원 수 :',len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [14]:
# 풀고자 하는 문제의 단어 집합 크기의 행과 100개의 열을 가지는 빈 행렬 생성
# 이후 빈 행렬에 사전 훈련된 임베딩 값 매핑
embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 100)


In [17]:
# 기존 데이터 단어-정수인코딩 확인
print(*tokenizer.word_index.items(), sep="\n")

('nice', 1)
('great', 2)
('best', 3)
('amazing', 4)
('stop', 5)
('lies', 6)
('pitiful', 7)
('nerd', 8)
('excellent', 9)
('work', 10)
('supreme', 11)
('quality', 12)
('bad', 13)
('highly', 14)
('respectable', 15)


In [19]:
# 'great' 맵핑 정수는 2
print('단어 great의 맵핑된 정수 :',tokenizer.word_index['great'])

단어 great의 맵핑된 정수 : 2


In [20]:
# 사전 훈련된 GloVe에서 'great'의 벡터값을 확인
print(embedding_dict['great'])

[-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0.61947
 -0.10146   -0.26301  

In [21]:
# 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [22]:
# 'great'의 벡터값이 의도한 인덱스의 위치에 삽입되었는지 확인
embedding_matrix[2]	# GloVe에서의 'great'의 벡터값과 일치!

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

In [24]:
# Embedding layer에 embedding_matrix를 초기값으로 설정

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim = 100	# 사전 훈련된 워드 임베딩은 100차원의 값

model = Sequential()
e = Embedding(
		vocab_size, output_dim, 
		weights=[embedding_matrix], # 사전훈련된 워드 임베딩을 가중치로
		input_length=max_len,
		trainable=False					# 사전훈련 워드 임베딩을 사용하므로 추가 훈련은 False
)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 1s 765ms/step - loss: 0.8275 - acc: 0.5714
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 0.8001 - acc: 0.5714
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 0.7735 - acc: 0.5714
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 0.7477 - acc: 0.5714
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.7228 - acc: 0.5714
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6987 - acc: 0.5714
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6756 - acc: 0.5714
Epoch 8/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6534 - acc: 0.7143
Epoch 9/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6321 - acc: 0.8571
Epoch 10/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6117 - acc: 0.8571
Epoch 11/100
1/1 [==============================] - 0s 9ms/step - loss: 0.

* 참고: 사전 훈련된 GloVe 임베딩 [예제](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

#### (2) 사전 훈련된 Word2Vec 사용하기

In [25]:
import gensim

# 구글의 사전 훈련된 Word2Vec 모델을 로드
urlretrieve(
		"https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz",
		filename="GoogleNews-vectors-negative300.bin.gz"
)
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

# 모델의 크기 확인: 사전 훈련된 워드 임베딩은 300차원
print('모델의 크기(shape) :',word2vec_model.vectors.shape)

모델의 크기(shape) : (3000000, 300)


In [28]:
# 풀고자 하는 문제의 단어 집합 크기의 행과 300개의 열을 가지는 빈 행렬 생성
# 이후 빈 행렬에 사전 훈련된 임베딩 값 매핑
embedding_matrix = np.zeros((vocab_size, 300))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 300)


In [29]:
#word2vec_model 특정 단어를 입력 => 해당 단어의 임베딩 벡터를 리턴
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
    		# 특정 단어의 임베딩 벡터가 없다면 None 리턴
        return None

In [31]:
# 기존 16개의 단어에 사전 훈련된 word2vec 임베딩 벡터 매핑
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [32]:
# 기존 16개 단어의 임베딩 행렬이 완성
print(word2vec_model['nice']) 

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [33]:
# 기존 단어 집합의 nice 정수 인코딩 확인
print('단어 nice의 맵핑된 정수 :', tokenizer.word_index['nice'])

단어 nice의 맵핑된 정수 : 1


In [34]:
# embedding_matrinx의 1번 인덱스('nice') 확인
print(embedding_matrix[1])

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [36]:
# Embedding에 사전 훈련된 embedding_matrix를 입력으로 넣어주고 모델을 학습

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input

model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
e = Embedding(
    vocab_size, 300, 
    input_length=max_len, 
    weights=[embedding_matrix], 
    trainable=False
)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 417ms/step - loss: 0.6495 - acc: 0.5714
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6327 - acc: 0.5714
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6164 - acc: 0.7143
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6007 - acc: 0.7143
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 0.5853 - acc: 0.7143
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5705 - acc: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.5561 - acc: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5422 - acc: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 0.5287 - acc: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5157 - acc: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 9ms/step - loss: 0.

* 참고: 사전 훈련된 워드 임베딩으로 의도 분류 실습 [예제](https://wikidocs.net/86083)

## 9. 엘모(Embeddings from Language Model, ELMo)

<div align="center">

![img](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSG5NF4QF_GKrEXKrGoonSYYLbNYs0RgV1VCQ&usqp=CAU)
</div>

* ELMo(Embeddings from Language Model)

  * 참고: ELMo 논문 [링크](https://aclweb.org/anthology/N18-1202)
  * 2018년에 제안된 새로운 워드 임베딩 방법론
  * Embeddings from Language Mode: 언어 모델로 하는 임베딩
  * **사전 훈련된 언어 모델(Pre-trained language model)**을 사용
  ```
  현재 텐서플로우 2.0에서는 TF-Hub의 ELMo를 사용할 수 없습니다. 
  사용하려면 텐서플로우 버전을 1버전으로 낮추어야 합니다. 
  Colab에서는 손쉽게 텐서플로우 버전을 1버전으로 설정할 수 있습니다.
  Colab에서 실습하시는 것을 권장드립니다. 
  ```

### 9.1. ELMo(Embeddings from Language Model)

* 동음이의어
  * Bank Account(은행 계좌) vs River Bank(강둑)
  * Word2Vec이나 GloVe 등으로 표현된 임베딩 벡터들은 이를 제대로 반영하지 못함
    * Bank는 전혀 다른 의미임에도 불구하고 두 가지 상황 모두에서 같은 벡터가 사용 ([0.2 0.8 -1.2])
* **문맥을 반영한 워드 임베딩(Contextualized Word Embedding)**
  * 문맥에 따라서 다르게 워드 임베딩하므로 자연어 처리의 성능 향상
  * 워드 임베딩 시 문맥을 고려해서 임베딩을 하겠다는 아이디어

### 9.2. biLM(Bidirectional Language Model)의 사전 훈련

* 깊은 단방향 RNN

  ![img](https://wikidocs.net/images/page/33930/deepbilm.PNG)

  * 은닉층이 2개인 일반적인 단방향 RNN 언어 모델의 언어 모델링
  * RNN의 은닉 상태의 ht의 값이 문장의 문맥 정보를 점차적으로 반영
  * ELMo는 순방향 RNN + 역방향 RNN 활용. **biLM(Bidirectional Language Model)**

* ELMo의 biLM

  ![img](https://wikidocs.net/images/page/33930/forwardbackwordlm2.PNG)

  * 기본적으로 다층 구조(Multi-layer)를 전제

  * biLM의 입력 벡터는 합성곱 신경망 방식의 **문자 임베딩**을 거친 단어 벡터 (임베딩 층 아님)

    * 문자 임베딩(character embedding)은 NLP를 위한 합성곱 신경망 챕터에서 다룸
    * 문자 임베딩은 임베딩층, Word2Vec 와는 다른 단어 벡터를 얻는 방식
    * 문맥과 상관없이 dog란 단어와 doggy란 단어의 연관성을 찾아낼 수 있음 (OOV에도 견고) 

  * **양방향 RNN**과 ELMo에서의 **biLM**은 다름

    * 양방향 RNN은 순방향 은닉 상태와 역방향의 은닉 상태를 연결(concatenate)하여 다음층의 입력으로 사용

      ![img](https://wikidocs.net/images/page/22886/rnn_image5_ver2.PNG)

    * biLM의 순방향 언어모델과 역방향 언어모델이라는 두 개의 언어 모델을 별개의 모델로 보고 학습

### 9.3. biLM의 활용

* 사전 훈련 모델 biLM(ELMo)을 통해 입력으로부터 단어를 임베딩하는 과정
  ![img](https://wikidocs.net/images/page/33930/playwordvector.PNG)

  * 해당 시점(time step)의 BiLM의 각 층 출력값을 로드
    * 출력값 이란 첫번째는 임베딩 층을, 나머지는 각 층의 은닉 상태
    * 각 층의 출력값이 가진 정보는 전부 서로 다른 종류의 정보. 이를 모두 활용
  * 순방향 언어 모델과 역방향 언어 모델의 각 층의 출력값을 연결(concatenate)
  * 추가 작업을 진행
    * 각 층의 연결된 출력값에 가중치를 곱한 값을 합하여 스칼라 매개변수를 곱함

**1) 각 층의 출력값을 연결(concatenate)한다.**

![img](https://wikidocs.net/images/page/33930/concatenate.PNG)

**2) 각 층의 출력값 별로 가중치를 준다.**

![img](https://wikidocs.net/images/page/33930/weight.PNG)

* 이 가중치를 여기서는 s1, s2, s3

**3) 각 층의 출력값을 모두 더한다.**

![img](https://wikidocs.net/images/page/33930/weightedsum.PNG)

* 2)번과 3)번의 단계를 요약하여 가중합(Weighted Sum)

**4) 벡터의 크기를 결정하는 스칼라 매개변수를 곱한다.**

![img](https://wikidocs.net/images/page/33930/scalarparameter.PNG)

* 스칼라 매개변수를 γ (감마)로 표현



* ELMo 표현(representation)

  * 완성된 벡터를 ELMo 표현이라고 함
  * ELMo 표현을 얻기 위한 과정 
  * ELMo를 입력으로 사용하고 수행하고 싶은 텍스트 분류, 질의 응답 시스템 등의 자연어 처리 작업을 진행

* ELMo를 활용한 모델링

  ![img](https://wikidocs.net/images/page/33930/elmorepresentation.PNG)

  * ELMo 표현을 기존의 임베딩 벡터와 함께 사용
  * 텍스트 분류 작업을 가정
    * GloVe와 같은 기존의 방법론을 사용한 임베딩 벡터를 준비
    * 여기에 ELMo 표현을 GloVe 임베딩 벡터와 연결(concatenate)해서 입력으로 사용
    * 이때 biLM의 가중치는 고정, 위에서 사용한 s1, s2, s3와 γ는 훈련 과정에서 학습

### 9.4. ELMo 표현을 사용해서 스팸 메일 분류하기
* 사전 훈련된 모델로부터 ELMo 표현을 사용해보는 정도로 예제

In [1]:
# 시작 전에 텐서플로우 버전을 1버전으로 설정 (colab) => 런타임 재시작
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [2]:
# 텐서플로우 허브로부터 다양한 사전 훈련된 모델(Pre-tained Model) 사용 가능
!pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

Using TensorFlow backend.


In [4]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [5]:
# 스팸 메일 분류 데이터를 다운(kaggle sms-spam-collection-dataset)
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
# 피쳐 v2열과 타겟 v1
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])

In [7]:
print(*X_data[:5], sep="\n")
print(y_data[:5])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
U dun say so early hor... U c already then say...
Nah I don't think he goes to usf, he lives around here though
[0, 0, 1, 0, 0]


In [8]:
# 훈련 데이터, 테스트 데이터 split
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)
print(n_of_test)

X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

5572
4457
1115


In [9]:
# ELMo와 설계한 모델을 연결하는 작업
# TF 허브에서 가져온 모델을 케라스에서 사용할 수 있도록 변환해주는 작업
def ELMoEmbedding(x):
    # 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]

In [10]:
# 모델 설계
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
# ELMo를 이용한 임베딩 층
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
# 256개의 뉴런이 있는 은닉층
hidden_layer = Dense(256, activation='relu')(embedding_layer)
# 1개의 뉴런을 통해 이진 분류. 시그모이드 활성화 함수
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
# 손실 함수는 binary_crossentropy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# 훈련 데이터에서는 정확도 96%. 놀랍다!
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

Epoch 1/1
4457/4457 [==============================] - 86s 19ms/step - loss: 0.1226 - accuracy: 0.9574


In [12]:
# 1번의 에포크에서 테스트 데이터 정확도 98%. 믿을 수 없는!
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 22s 20ms/step

 테스트 정확도: 0.9767


## 10. 임베딩 벡터의 시각화(Embedding Visualization)

* 구글에서 임베딩 프로젝터(embedding projector)라는 데이터 시각화 도구를 지원
* 학습한 임베딩 벡터들을 시각화해보는 실습

* 참고: 임베딩 프로젝터 논문 [링크](https://arxiv.org/pdf/1611.05469v1.pdf )

### 10.1. 워드 임베딩 모델로부터 2개의 tsv 파일 생성하기

* 시각화를 위해서는 학습이 끝난 모델이 파일로 저장되어 있어야 함

* 아래 커맨드를 통해 시각화에 필요한 파일들을 생성

  ```python
  # 이전 챕터에서 저장한 영어 Word2Vec 모델 사용
  # !python -m gensim.scripts.word2vec2tensor --input 모델이름 --output 모델이름
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v
  ```

* eng_w2v 외 두 개의 파일이 추가

  ![img](https://wikidocs.net/images/page/50704/eng_w2v.PNG)

  * eng_w2v_metadata.tsv
  * eng_w2v_tensor.tsv 
  * 이 두 개 파일이 임베딩 벡터 시각화를 위해 사용할 파일


### 10.2. 임베딩 프로젝터를 사용하여 시각화하기

1. 구글 임베딩 프로젝터에 [접속](https://projector.tensorflow.org/ )

2. 좌측 상단을 보면 Load 버튼 클릭

   ![img](https://wikidocs.net/images/page/50704/embedding_projector.PNG)



3. 두 개의 choose file 버튼 생성

   ![img](https://wikidocs.net/images/page/50704/embedding_projector2.PNG)

   * Load a TSV file of vectors => Choose file => *_tensor.tsv 업로드
   * Load a TSV file of metadata => Choose file => *_metadata.tsv 업로드

4. 임베딩 프로젝터에 학습했던 워드 임베딩 모델이 시각화

   ![img](https://wikidocs.net/images/page/50704/man.PNG)

5. 임베딩 프로젝터의 다양한 기능 사용 가능

   * 복잡한 데이터를 차원을 축소하여 시각화 할 수 있도록 도와주는 PCA, t-SNE 등을 제공
   * 위의 그림은 'man' 이라는 단어를 선택하고, 코사인 유사도를 기준으로 가장 유사한 상위 10개 벡터들을 표시